In [1]:
import os
import sys
import time
import random
import numpy as np
# import pandas as pd
# import keras
import tensorflow as tf
import pickle

import skimage
import skimage.io
import skimage.transform
# from skimage.viewer import ImageViewer
from matplotlib.pyplot import imshow
%matplotlib inline

from PIL import Image
import scipy
from sklearn.utils import shuffle

from sklearn.metrics import accuracy_score

from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def leaky_relu(features, alpha=0.2, name=None):
    """Compute the Leaky ReLU activation function.
    "Rectifier Nonlinearities Improve Neural Network Acoustic Models"
    AL Maas, AY Hannun, AY Ng - Proc. ICML, 2013
    http://web.stanford.edu/~awni/papers/relu_hybrid_icml2013_final.pdf
    Args:
    features: A `Tensor` representing preactivation values.
    alpha: Slope of the activation function at x < 0.
    name: A name for the operation (optional).
    Returns:
    The activation value.
    """
    with ops.name_scope(name, "LeakyRelu", [features, alpha]):
        features = ops.convert_to_tensor(features, name="features")
        alpha = ops.convert_to_tensor(alpha, name="alpha")
        return math_ops.maximum(alpha * features, features)

## argument setting

In [2]:
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)

# def init():
#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     session = tf.Session(config=config)
#     keras.backend.tensorflow_backend.set_session(session)
# init()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

img_path = './data'
len_img_all = 33431
condition = True
load_data = True
tag_img_only = True
batch_size = 16
iteration = 100000
D_ITER = 1
G_ITER = 1
d_lda = 10

toy_test = False
if toy_test :
    len_img_all = 512
    load_data = False
if tag_img_only :
    len_img_all = 11568
output_str = 'gan9_bs16_HEclass_original'
load_model = False

pretrain_hair = True

## preprocessing

In [3]:
def img_prep(save=False, lst_null=None) :
    print ('img_prep is runing')
    if tag_img_only :
        lst_img = []
        for img_i in range(len_img_all) :
            if img_i in lst_null :
                continue
            img_temp = skimage.io.imread('{}/faces/{}.jpg'.format(img_path,img_i))
            img_temp = skimage.transform.resize(img_temp, (64,64))
            lst_img += [img_temp]
    else :
        ary_img = np.zeros((len_img_all,64,64,3))
        for img_i in range(len_img_all) :
            img_temp = skimage.io.imread('{}/faces/{}.jpg'.format(img_path,img_i))
            img_temp = skimage.transform.resize(img_temp, (64,64))
            ary_img[img_i] = img_temp
    if save :
        if tag_img_only :
            print ('./data_pp/ary_img_withtag is saving')
            ary_img = np.asarray(lst_img)
            np.save('./data_pp/ary_img_withtag', ary_img)
        else :
            np.save('./data_pp/ary_img', ary_img)
            
    print ('len_ary_img : {}'.format(len(ary_img)))
    return ary_img
    
def tag_prep(save=False) :
    # 1,6
    # 2,4,12
    # 3,5,9
    # 7,10
    lst_hair = ['null', 'orange hair', 'white hair', 'aqua hair', 'gray hair',
                'green hair', 'red hair', 'purple hair', 'pink hair',
                'blue hair', 'black hair', 'brown hair', 'blonde hair']
    lst_eyes = ['null', 'gray eyes', 'black eyes', 'orange eyes',
                'pink eyes', 'yellow eyes', 'aqua eyes', 'purple eyes',
                'green eyes', 'brown eyes', 'red eyes', 'blue eyes']
    
    h_n = 0
    e_n = 0  
    lst_null = []
    if tag_img_only :
        lst_tag_hair = []
        lst_tag_eyes = []
        with open('{}/tags_clean.csv'.format(img_path), 'r') as f :
            for line_i,line in enumerate(f.readlines()[:len_img_all]) :
        #         sys.stdout.write("\r{}\t".format(line_i))
        #         sys.stdout.flush()

                FLAG_h = False
                FLAG_e = False
                FLAG_double_tag = False
                for h_i,h in enumerate(lst_hair) :
                    if h in line :
                        if FLAG_h :
                            # double tag
                            lst_tag_hair = lst_tag_hair[:-1]
                            lst_null += [line_i]
                            FLAG_double_tag = True
                            break
                        FLAG_h = 1
                        lst_tag_hair += [h_i]
                        h_n += 1
                if not FLAG_h :
                    lst_null += [line_i]
                    continue
                if FLAG_double_tag :
                    continue
                for e_i,e in enumerate(lst_eyes) :
                    if e in line :
                        if FLAG_e :
                            # double tag
                            lst_tag_eyes = lst_tag_eyes[:-1]
                            lst_null += [line_i]
                            lst_tag_hair = lst_tag_hair[:-1]
                            break
                        FLAG_e = 1
                        lst_tag_eyes += [e_i]
                        e_n += 1
                if not FLAG_e :
                    lst_null += [line_i]
                    lst_tag_hair = lst_tag_hair[:-1]
            print (len(lst_tag_hair))
            print (len(lst_tag_eyes))
            print ('number of hair tag : {}'.format(h_n))
            print ('number of eyes tag : {}'.format(e_n))
            print ('number of total img : {}'.format(line_i+1))
            print ('number of total both tag : {}'.format(len(lst_tag_hair)))
            print ('number of drop tag : {}'.format(len(lst_null)))
            assert len(lst_tag_hair) == len(lst_tag_eyes), 'error'
            print ('tag preprocessing done')
            
    else :
        ary_tag_hair = np.zeros((len_img_all))
        ary_tag_eyes = np.zeros((len_img_all))
        with open('{}/tags_clean.csv'.format(img_path), 'r') as f :
            for line_i,line in enumerate(f.readlines()[:len_img_all]) :
        #         sys.stdout.write("\r{}\t".format(line_i))
        #         sys.stdout.flush()

                flag_h = 0
                flag_e = 0
                for h_i,h in enumerate(lst_hair) :
                    if h in line :
                        ary_tag_hair[line_i] = h_i
                        h_n += 1
                for e_i,e in enumerate(lst_eyes) :
                    if e in line :
                        ary_tag_eyes[line_i] = e_i
                        e_n += 1
            assert len(ary_tag_hair) == len(ary_tag_eyes), 'error'
            print ('number of hair tag : {}'.format(h_n))
            print ('number of eyes tag : {}'.format(e_n))
            print ('number of total img : {}'.format(line_i))
            print ('tag preprocessing done')
        
    if save :
        if not os.path.isdir('./data_pp') :
            os.makedirs('./data_pp')
        if tag_img_only :
            print ('./data_pp/ary_tag_hair_withtag is saving')
            ary_tag_hair = np.asarray(lst_tag_hair)
            ary_tag_eyes = np.asarray(lst_tag_eyes)
            np.save('./data_pp/ary_tag_hair_withtag', ary_tag_hair)
            np.save('./data_pp/ary_tag_eyes_withtag', ary_tag_eyes)
        else :
            print ('./data_pp/ary_tag_hair is saving')
            np.save('./data_pp/ary_tag_hair', ary_tag_hair)
            np.save('./data_pp/ary_tag_eyes', ary_tag_eyes)
    
    return ary_tag_hair, ary_tag_eyes, lst_null
    
def preprocess(show=True, save=False, load=False) :
    if load :
        if tag_img_only :
            ary_tag_hair = np.load('./data_pp/ary_tag_hair_withtag.npy')
            ary_tag_eyes = np.load('./data_pp/ary_tag_eyes_withtag.npy')
            ary_img = np.load('./data_pp/ary_img_withtag.npy')
        else :
            ary_tag_hair = np.load('./data_pp/ary_tag_hair.npy')
            ary_tag_eyes = np.load('./data_pp/ary_tag_eyes.npy')
            ary_img = np.load('./data_pp/ary_img.npy')
    else :
        ary_tag_hair, ary_tag_eyes, lst_null = tag_prep(save=save)
        ary_img = img_prep(save=save, lst_null=lst_null)

    if show :
        print ('ary_tag_hair shape : {}'.format(ary_tag_hair.shape))
        print ('ary_tag_eyes shape : {}'.format(ary_tag_eyes.shape))
        print ('ary_img shape : {}'.format(ary_img.shape))
#         imshow(ary_img[1])
    
    return ary_tag_hair, ary_tag_eyes, ary_img

ary_tag_hair, ary_tag_eyes, ary_img = preprocess(show=True,save=False,load=load_data)

ary_tag_hair shape : (11568,)
ary_tag_eyes shape : (11568,)
ary_img shape : (11568, 64, 64, 3)


## model

In [4]:
class GAN_no_condition(object):
    def __init__(self) :
        self.lr = 0.0001
        self.momentum = 0.5
        self.bs = batch_size # batch size is m of paper
        self.bs_pre = 64
        self.epoch = 10000
        self.hair_n = 13
        self.eyes_n = 12
        self.lda = d_lda
        self.epsilon = 0.5
        self.activation = leaky_relu
        self.initializer = tf.contrib.keras.initializers.he_normal()
        
    def build_G_net(self) :
        with tf.variable_scope('G') as g_scope:
#             self.G_in_hair = tf.placeholder(tf.int32, shape=[None])
#             self.G_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.G_in_noise = tf.placeholder(tf.float32, shape=[None,100])
#             self.G_in_noise = tf.distributions.Normal(loc=0., scale=1.).sample([self.bs,100])
            
#             G_H_onehot = tf.one_hot(self.G_in_hair, self.hair_n)
#             G_E_onehot = tf.one_hot(self.G_in_eyes, self.eyes_n)
#             g = tf.concat([G_H_onehot, G_E_onehot, self.G_in_noise], axis=1)
            
#             g = keras.layers.Dense(4*4*1024, activation='linear')(g)
            g = tf.layers.dense(self.G_in_noise,4*4*1024,activation=None)
            g = tf.reshape(g,(-1,4,4,1024))
#             g = self.activation(g)
            
            g = tf.layers.conv2d_transpose(g, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.conv2d_transpose(g, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.conv2d_transpose(g, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            self.g = tf.layers.conv2d_transpose(g, filters=3, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            
            ####
#             d = tf.layers.conv2d(self.D_img, filters=128, kernel_size=(5,5), strides=(2,2), 
#                                  padding='same', activation=self.activation, name='d_c1', reuse=None)
#             d = tf.layers.conv2d(d, filters=256, kernel_size=(5,5), strides=(2,2), 
#                                  padding='same', activation=self.activation, name='d_c2', reuse=None)
#             print (d)
            
        
    def build_D_net(self) :
        with tf.variable_scope('D') as d_scope:
            self.D_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3])

            self.D_in_img_right = self.D_in_img[:self.bs]
            self.D_in_img_fake = self.D_in_img[self.bs:]
            
            use_epsilon_uniform = 1
            if use_epsilon_uniform :
                epsilon = np.random.random_sample()
                self.D_x_hat_img = epsilon*self.D_in_img_right + (1-epsilon)*self.D_in_img_fake
            else :
                self.D_x_hat_img = self.epsilon*self.D_in_img_right + (1-self.epsilon)*self.D_in_img_fake
                
            self.D_img = tf.concat([self.D_in_img_right,self.D_in_img_fake,self.D_x_hat_img], axis=0)
            
            d = tf.layers.conv2d(self.D_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=None)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=None)
            d = tf.layers.conv2d(d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=None)
            d = tf.layers.conv2d(d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=None)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(1,1), strides=(1,1), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c5', reuse=None)
            d = tf.reshape(d, [-1, 4*4*256]) 
            self.d_temp = tf.layers.dense(d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=None)
            
            self.d_right, self.d_fake, self.d_x_hat = tf.split(self.d_temp, num_or_size_splits=3, axis=0)
#             self.d_right, self.d_fake_temp = tf.split(self.d_temp, num_or_size_splits=2, axis=0)
#             self.d_fake, self.d_x_hat = tf.split(self.d_fake_temp, num_or_size_splits=2, axis=0)
            
#             ### wgan loss
#             self.d = tf.concat([self.d_right, self.d_fake], axis=0)
            
            ### gradient penalty
#             gradient_temp = tf.gradients(self.d_x_hat,[self.D_x_hat_img,self.D_x_hat_hair,self.D_x_hat_eyes])
            gradient_temp = tf.gradients(self.d_x_hat,self.D_x_hat_img)
            print (gradient_temp)
#             self.gradient_penalty = tf.maximum(tf.constant(0.0, shape=[self.bs,1]), tf.sqrt(tf.square(gradient_temp)) - tf.constant(1.0, shape=[self.bs,1]))
            self.gradient_penalty = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_temp), axis=1)) - 1)
            self.gradient_penalty = self.lda * tf.reduce_mean(self.gradient_penalty)
            
            ### final loss
            self.d_loss = - (tf.reduce_mean(self.d_right) - tf.reduce_mean(self.d_fake) - self.gradient_penalty)
            
            
            self.d_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.d_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.d_loss, var_list=self.d_train_vars)
            
            ###
            ### G loss
            ###
            self.G_D_in_img = self.g
            
            g_d = tf.layers.conv2d(self.G_D_in_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=True)
            
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(1,1), strides=(1,1), 
                                 padding='same', activation=self.activation, name='d_c5', kernel_initializer=self.initializer, reuse=True)
            g_d = tf.reshape(g_d, [-1, 4*4*256]) 
            self.g_d_temp = tf.layers.dense(g_d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=True)
            
            self.gd_loss = - tf.reduce_mean(self.g_d_temp)
            
            self.gd_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"G/")
            print (self.gd_train_vars)
            self.gd_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.gd_loss, var_list=self.gd_train_vars)
            
    def build_net(self) :
        self.build_G_net()
        self.build_D_net()
        
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        tf.summary.FileWriter("logs/", self.sess.graph)


In [ ]:
class GAN(object):
    def __init__(self) :
        self.lr = 0.0001
        self.lr_pre = 0.0001
        self.momentum = 0.5
        self.bs = batch_size # batch size is m of paper
        self.bs_pre = 128
        self.epoch = 10000
        self.hair_n = 13
        self.eyes_n = 12
        self.lda = d_lda
        self.epsilon = 0.5
        self.activation = leaky_relu
        self.initializer = tf.contrib.keras.initializers.he_normal()
        
#     def build_hair_net(self) :
#         with tf.variable_scope('hair') as hair_scope:
#             self.hair_in_hair = tf.placeholder(tf.int32, shape=[None])
#             self.hair_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 
            
#             hair_H_onehot = tf.one_hot(self.hair_in_hair, self.hair_n)
#             hair_temp = tf.layers.batch_normalization(self.hair_in_img, name='hair_b0', reuse=None)
#             hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
#                                  padding='same', activation=self.activation, name='hair_c1', reuse=None)
#             hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b1', reuse=None)
#             hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
#                                  padding='same', activation=self.activation, name='hair_c2', reuse=None)
#             hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b2', reuse=None)
#             hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
#                                  padding='same', activation=self.activation, name='hair_c3', reuse=None)
#             hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b3', reuse=None)
#             hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
#                                  padding='same', activation=self.activation, name='hair_c4', reuse=None)
#             hair_temp = tf.reshape(hair_temp, [-1, 4*4*32]) 
#             self.hair = tf.layers.dense(hair_temp, self.hair_n ,activation=tf.nn.softmax, name='hair_s2', kernel_initializer=self.initializer, reuse=None)
            
#             self.hair_loss = tf.reduce_mean(-tf.reduce_sum(hair_H_onehot * tf.log(self.hair)))
#             self.hair_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"hair/")
#             self.hair_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.hair_loss, var_list=self.hair_train_vars)
            
    def build_eyes_net(self) :
        with tf.variable_scope('eyes') as eyes_scope:
            self.eyes_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.eyes_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 

            eyes_E_onehot = tf.one_hot(self.eyes_in_eyes, self.eyes_n)
            eyes_temp = tf.layers.batch_normalization(self.eyes_in_img, name='eyes_b0', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c1', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b1', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c2', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b2', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c3', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b3', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c4', reuse=None)
            eyes_temp = tf.reshape(eyes_temp, [-1, 4*4*32]) 
            self.eyes = tf.layers.dense(eyes_temp, self.eyes_n ,activation=tf.nn.softmax, name='eyes_s2', kernel_initializer=self.initializer, reuse=None)

            self.eyes_loss = tf.reduce_mean(-tf.reduce_sum(eyes_E_onehot * tf.log(self.eyes), reduction_indices=[1]))
            self.eyes_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"eyes/")
            self.eyes_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.eyes_loss, var_list=self.eyes_train_vars)
            
            
    def build_G_net(self) :
        with tf.variable_scope('G') as g_scope:
            self.G_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.G_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.G_in_noise = tf.placeholder(tf.float32, shape=[None,100])
            
            self.G_H_onehot = tf.one_hot(self.G_in_hair, self.hair_n)
            self.G_E_onehot = tf.one_hot(self.G_in_eyes, self.eyes_n)
            g = tf.concat([self.G_H_onehot, self.G_E_onehot, self.G_in_noise], axis=1)
            g = tf.layers.dense(g,4*4*1024,activation=None)
            g = tf.reshape(g,(-1,4,4,1024))
            
            g = tf.layers.conv2d_transpose(g, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            g = tf.layers.conv2d_transpose(g, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            g = tf.layers.conv2d_transpose(g, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            self.g = tf.layers.conv2d_transpose(g, filters=3, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=tf.nn.tanh)
        
    def build_D_net(self) :

        with tf.variable_scope('D') as d_scope:

            ### include right and fake1
            self.D_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.D_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.D_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3])
#             self.D_in_img = self.D_in_img*2 - 1
            
            D_H_onehot = tf.one_hot(self.D_in_hair, self.hair_n)
            D_E_onehot = tf.one_hot(self.D_in_eyes, self.eyes_n)
            
            self.D_in_hair_right = D_H_onehot[:self.bs]
            self.D_in_eyes_right = D_E_onehot[:self.bs]
            self.D_in_img_right = self.D_in_img[:self.bs]
            
            self.D_in_hair_fake = D_H_onehot[self.bs:]
            self.D_in_eyes_fake = D_E_onehot[self.bs:]
            self.D_in_img_fake = self.D_in_img[self.bs:]
            
            use_epsilon_uniform = 1
            if use_epsilon_uniform :
                self.epsilon = tf.placeholder(tf.float32, shape=[1])
                self.D_x_hat_hair = self.epsilon*self.D_in_hair_right + (1-self.epsilon)*self.D_in_hair_fake
                self.D_x_hat_eyes = self.epsilon*self.D_in_eyes_right + (1-self.epsilon)*self.D_in_eyes_fake
                self.D_x_hat_img = self.epsilon*self.D_in_img_right + (1-self.epsilon)*self.D_in_img_fake
            else :
                self.D_x_hat_hair = self.epsilon*self.D_in_hair_right + (1-self.epsilon)*self.D_in_hair_fake
                self.D_x_hat_eyes = self.epsilon*self.D_in_eyes_right + (1-self.epsilon)*self.D_in_eyes_fake
                self.D_x_hat_img = self.epsilon*self.D_in_img_right + (1-self.epsilon)*self.D_in_img_fake
                
            self.D_img = tf.concat([self.D_in_img_right,self.D_in_img_fake,self.D_x_hat_img], axis=0)
            self.D_hair = tf.concat([self.D_in_hair_right,self.D_in_hair_fake,self.D_x_hat_hair], axis=0)
            self.D_eyes = tf.concat([self.D_in_eyes_right,self.D_in_eyes_fake,self.D_x_hat_eyes], axis=0)
            
            d = tf.layers.conv2d(self.D_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=None)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=None)
            d = tf.layers.conv2d(d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=None)
            d = tf.layers.conv2d(d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=None)
            
            D_tag_in = tf.concat([self.D_hair, self.D_eyes], axis=1)
            D_tag_in = tf.layers.dense(D_tag_in,4*4*1024,activation=self.activation, name='d_d1', kernel_initializer=self.initializer, reuse=None)
            D_tag_in = tf.reshape(D_tag_in,(-1,4,4,1024))
            
            d = tf.concat([d, D_tag_in], axis=3)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(1,1), strides=(1,1), 
                                 padding='same', activation=self.activation, name='d_c5', kernel_initializer=self.initializer, reuse=None)
            d = tf.reshape(d, [-1, 4*4*256]) 
            self.d_temp = tf.layers.dense(d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=None)
        
            self.d_right, self.d_fake, self.d_x_hat = tf.split(self.d_temp, num_or_size_splits=3, axis=0)
#             self.d_right, self.d_fake_temp = tf.split(self.d_temp, num_or_size_splits=2, axis=0)
#             self.d_fake, self.d_x_hat = tf.split(self.d_fake_temp, num_or_size_splits=2, axis=0)
            
            ### gradient penalty
#             gradient_temp = tf.gradients(self.d_x_hat,[self.D_x_hat_img,self.D_x_hat_hair,self.D_x_hat_eyes])
            gradient_img_temp = tf.gradients(self.d_x_hat,self.D_x_hat_img)
            gradient_hair_temp = tf.gradients(self.d_x_hat,self.D_x_hat_hair)
            gradient_eyes_temp = tf.gradients(self.d_x_hat,self.D_x_hat_eyes)
#             self.gradient_penalty = tf.maximum(tf.constant(0.0, shape=[self.bs,1]), tf.sqrt(tf.square(gradient_temp)) - tf.constant(1.0, shape=[self.bs,1]))
            self.gradient_penalty_img = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_img_temp), axis=1)) - 1)
#             self.gradient_penalty_img = self.lda * tf.reduce_mean(self.gradient_penalty_img)
            self.gradient_penalty_hair = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_hair_temp), axis=1)) - 1)
#             self.gradient_penalty_hair = self.lda * tf.reduce_mean(self.gradient_penalty_hair)
            self.gradient_penalty_eyes = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_eyes_temp), axis=1)) - 1)
#             self.gradient_penalty_eyes = self.lda * tf.reduce_mean(self.gradient_penalty_eyes)
            self.gradient_penalty = self.lda * (tf.reduce_mean(self.gradient_penalty_img)
                                                + tf.reduce_mean(self.gradient_penalty_hair)/10
                                                + tf.reduce_mean(self.gradient_penalty_eyes)/10)
            
            ### final loss
            self.d_loss = -(tf.reduce_mean(self.d_right) - tf.reduce_mean(self.d_fake) - self.gradient_penalty)
            self.d_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.d_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.d_loss, var_list=self.d_train_vars)
            
            ### G_D loss
            self.G_D_in_hair = self.G_in_hair
            self.G_D_in_eyes = self.G_in_eyes
            self.G_D_in_img = self.g
            
            self.G_D_eyes = tf.one_hot(self.G_D_in_eyes, self.eyes_n)
            self.G_D_hair = tf.one_hot(self.G_D_in_hair, self.hair_n)
            
            g_d = tf.layers.conv2d(self.G_D_in_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=True)
            
            G_D_tag_in = tf.concat([self.G_D_hair, self.G_D_eyes], axis=1)
            G_D_tag_in = tf.layers.dense(G_D_tag_in,4*4*1024,activation=self.activation, name='d_d1', kernel_initializer=self.initializer, reuse=True)
            G_D_tag_in = tf.reshape(G_D_tag_in,(-1,4,4,1024))
            
            g_d = tf.concat([g_d, G_D_tag_in], axis=3)
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(1,1), strides=(1,1), 
                                 padding='same', activation=self.activation, name='d_c5', kernel_initializer=self.initializer, reuse=True)
            g_d = tf.reshape(g_d, [-1, 4*4*256]) 
            self.g_d_temp = tf.layers.dense(g_d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=True)
            
            self.gd_loss = - tf.reduce_mean(self.g_d_temp)
            
            #
            # HE classification
            #
            
            # hair classification pretrained
            self.hair_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.hair_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 

            hair_H_onehot = tf.one_hot(self.hair_in_hair, self.hair_n)
            hair_temp = tf.layers.batch_normalization(self.hair_in_img, name='hair_b0', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c1', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b1', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c2', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b2', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c3', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b3', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c4', reuse=None)
            hair_temp = tf.reshape(hair_temp, [-1, 4*4*32]) 
            self.hair = tf.layers.dense(hair_temp, self.hair_n ,activation=tf.nn.softmax, name='hair_s2', kernel_initializer=self.initializer, reuse=None)

            self.hair_loss = tf.reduce_mean(-tf.reduce_sum(hair_H_onehot * tf.log(self.hair), axis=1))
            self.hair_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.hair_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.hair_loss, var_list=self.hair_train_vars)
            
            # eyes classification pretrained
            self.eyes_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.eyes_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 

            eyes_E_onehot = tf.one_hot(self.eyes_in_eyes, self.eyes_n)
            eyes_temp = tf.layers.batch_normalization(self.eyes_in_img, name='eyes_b0', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c1', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b1', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c2', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b2', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c3', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b3', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c4', reuse=None)
            eyes_temp = tf.reshape(eyes_temp, [-1, 4*4*32]) 
            self.eyes = tf.layers.dense(eyes_temp, self.eyes_n ,activation=tf.nn.softmax, name='eyes_s2', kernel_initializer=self.initializer, reuse=None)

            self.eyes_loss = tf.reduce_mean(-tf.reduce_sum(eyes_E_onehot * tf.log(self.eyes), axis=1))
            self.eyes_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.eyes_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.eyes_loss, var_list=self.eyes_train_vars)
            
            ### hair classification
            gh_temp = tf.layers.batch_normalization(self.g, name='hair_b0', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c1', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b1', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c2', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b2', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c3', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b3', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c4', reuse=True)
            gh_temp = tf.reshape(gh_temp, [-1, 4*4*32]) 
            self.gh = tf.layers.dense(gh_temp, self.hair_n ,activation=tf.nn.softmax, name='hair_s2', kernel_initializer=self.initializer, reuse=True)
            
            self.gh_loss = tf.reduce_mean(-tf.reduce_sum(self.G_H_onehot * tf.log(self.gh), axis=1))
            
            ### eyes classification
            ge_temp = tf.layers.batch_normalization(self.g, name='eyes_b0', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c1', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b1', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c2', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b2', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c3', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b3', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c4', reuse=True)
            ge_temp = tf.reshape(ge_temp, [-1, 4*4*32]) 
            self.ge = tf.layers.dense(ge_temp, self.eyes_n ,activation=tf.nn.softmax, name='eyes_s2', kernel_initializer=self.initializer, reuse=True)

            self.ge_loss = tf.reduce_mean(-tf.reduce_sum(self.G_E_onehot * tf.log(self.ge), axis=1))
            
            
            
            self.gd_loss_final = 1*self.gd_loss + 1*self.gh_loss + 1*self.ge_loss
            
            self.gd_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"G/")
            self.gd_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.gd_loss_final, var_list=self.gd_train_vars)
            
    def build_net(self) :
#         self.build_hair_net()
#         self.build_eyes_net()
        
        self.build_G_net()
        self.build_D_net()
        
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        tf.summary.FileWriter("logs/", self.sess.graph)


## training

In [ ]:
def training() :
    tf.reset_default_graph()
    ary_tag_hair, ary_tag_eyes, ary_img = preprocess(save=False,load=load_data)
    if condition :
        gan = GAN()
        gan.build_net()
        if load_model :
            pass
        pretrain_hair = True
        if pretrain_hair :
            for i in range(10) :
                print (i)
                ary_tag_hair_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_hair[:-1000], ary_tag_eyes[:-1000], ary_img[:-1000], random_state=i)
                ary_img_sh = ary_img_sh*2 -1
                b_i = 0
                while b_i+gan.bs_pre < len_img_all-1000 :
                    sys.stdout.write("\r{}\t".format(b_i))
                    sys.stdout.flush()
                    b_tag_hair = ary_tag_hair_sh[b_i:b_i+gan.bs_pre]
                    b_img = ary_img_sh[b_i:b_i+gan.bs_pre]
                    
                    _, loss = gan.sess.run([gan.hair_train,gan.hair_loss], feed_dict={gan.hair_in_hair:b_tag_hair,
                                                            gan.hair_in_img:b_img})
#                     print (loss)
                    b_i += gan.bs_pre
                ### validation    
                y_pred = np.argmax(gan.sess.run(gan.hair, feed_dict={gan.hair_in_img:ary_img[-500:]}), axis=1)
                y_true = ary_tag_hair[-500:]
#                 print ('\n')
#                 print (y_pred)
#                 print (y_true)
                print ('acc : {}'.format(accuracy_score(y_true,y_pred)))

        pretrain_eyes = True
        if pretrain_eyes :
            for i in range(10) :
                print (i)
                ary_tag_eyes_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_eyes[:-1000], ary_tag_eyes[:-1000], ary_img[:-1000], random_state=i)
                ary_img_sh = ary_img_sh*2 -1
                b_i = 0
                while b_i+gan.bs_pre < len_img_all-1000 :
                    sys.stdout.write("\r{}\t".format(b_i))
                    sys.stdout.flush()
                    b_tag_eyes = ary_tag_eyes_sh[b_i:b_i+gan.bs_pre]
                    b_img = ary_img_sh[b_i:b_i+gan.bs_pre]
                    
                    _, loss = gan.sess.run([gan.eyes_train,gan.eyes_loss], feed_dict={gan.eyes_in_eyes:b_tag_eyes,
                                                            gan.eyes_in_img:b_img})
#                     print (loss)
                    b_i += gan.bs_pre
                ### validation    
                y_pred = np.argmax(gan.sess.run(gan.eyes, feed_dict={gan.eyes_in_img:ary_img[-500:]}), axis=1)
                y_true = ary_tag_eyes[-500:]
#                 print ('\n')
#                 print (y_pred)
#                 print (y_true)
                print ('acc : {}'.format(accuracy_score(y_true,y_pred)))
    else :
        gan = GAN_no_condition()
        gan.build_net()
    
    lst_loss_his_d = []
    lst_loss_his_g = []
    for i in range(iteration) :
        print(i)
        ary_tag_hair_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_hair, ary_tag_eyes, ary_img, random_state=i)
        gen_tag_hair = np.random.randint(13, size=int(len_img_all/2+1))
        gen_tag_eyes = np.random.randint(12, size=int(len_img_all/2+1))
        ary_img_sh = ary_img_sh*2 -1
        
        b_i = 0
        while b_i+gan.bs <= len_img_all :
            b_tag_hair_right = ary_tag_hair_sh[b_i:b_i+gan.bs]
            b_tag_eyes_right = ary_tag_eyes_sh[b_i:b_i+gan.bs]
            b_img_right = ary_img_sh[b_i:b_i+gan.bs]
            
            # fake1 (right img wrong text)
            b_tag_hair_fake1 = np.random.randint(1,13, size=int(gan.bs/4))
            b_tag_eyes_fake1 = np.random.randint(1,12, size=int(gan.bs/4))
            b_img_fake1 = np.copy(b_img_right[:int(gan.bs/4)])
            for ii in range(int(gan.bs/4)) :
                while b_tag_hair_fake1[ii] == b_tag_hair_right[ii] :
                    b_tag_hair_fake1[ii] = random.randint(0,13)
                while b_tag_eyes_fake1[ii] == b_tag_eyes_right[ii] :
                    b_tag_eyes_fake1[ii] = random.randint(0,12)
                    
            # fake2 (wrong img right text)
            b_tag_hair_fake2 = np.copy(b_tag_hair_right[:int(gan.bs/4)])
            b_tag_eyes_fake2 = np.copy(b_tag_eyes_right[:int(gan.bs/4)])
            lst_random_num = random.sample(range(len_img_all),k=int(gan.bs/4))
            b_img_fake2 = np.copy(ary_img_sh[lst_random_num])
            
            # fake3 (generate img right text)
            b_tag_hair_fake3 = gen_tag_hair[int(b_i/2):int(b_i/2+gan.bs/2)]
            b_tag_eyes_fake3 = gen_tag_eyes[int(b_i/2):int(b_i/2+gan.bs/2)]
            ary_temp = np.random.normal(0,1,[b_tag_eyes_fake3.shape[0],100])

            b_img_fake3 = gan.sess.run(gan.g, feed_dict={gan.G_in_hair:b_tag_hair_fake3, 
                                                         gan.G_in_eyes:b_tag_eyes_fake3, 
                                                         gan.G_in_noise:ary_temp})
            
            
            # update D
            b_tag_hair = np.concatenate((b_tag_hair_right,b_tag_hair_fake1,b_tag_hair_fake2,b_tag_hair_fake3), axis=0)
            b_tag_eyes = np.concatenate((b_tag_eyes_right,b_tag_eyes_fake1,b_tag_eyes_fake2,b_tag_eyes_fake3), axis=0)
            b_img = np.concatenate((b_img_right,b_img_fake1,b_img_fake2,b_img_fake3), axis=0)
            b_epsilon = np.random.rand(1,)
            for i2 in range(D_ITER) :
                _, loss_D = gan.sess.run([gan.d_train,gan.d_loss], feed_dict={gan.D_in_hair:b_tag_hair, 
                                                          gan.D_in_eyes:b_tag_eyes, 
                                                          gan.D_in_img:b_img,
                                                          gan.epsilon:b_epsilon})
            
                print ('D loss : {}'.format(loss_D))
            
            # update G
            b_tag_hair_g = gen_tag_hair[int(b_i/2):int(b_i/2+gan.bs/2)]
            b_tag_eyes_g = gen_tag_eyes[int(b_i/2):int(b_i/2+gan.bs/2)]
            ary_temp_g = np.random.normal(0,1,[b_tag_hair_g.shape[0],100])
            for i3 in range(G_ITER) :
                _, loss_G = gan.sess.run([gan.gd_train,gan.gd_loss], feed_dict={gan.G_in_hair:b_tag_hair_g, 
                                                         gan.G_in_eyes:b_tag_eyes_g, 
                                                         gan.G_in_noise:ary_temp_g})
    
                print ('G loss : {}'.format(loss_G))
            
            
            if b_i % 6400 == 0 :
                lst_loss_his_d += [loss_D]
                lst_loss_his_g += [loss_G]
                print ('saving model...')
                if not os.path.isdir('./model_tf') :
                    os.mkdir('./model_tf')
                if not os.path.isdir('./record') :
                    os.mkdir('./record')
                if not os.path.isdir('./img') :
                    os.mkdir('./img')
                k = 0
                while 1 :
                    if os.path.isfile('./model_tf/model_{}_{}.ckpt.meta'.format(output_str, k)) :
                        k += 1
                    else :
                        break
                save_path = gan.saver.save(gan.sess, './model_tf/model_{}_{}.ckpt'.format(output_str, k))
                with open('./record/loss_g_{}_{}.pkl'.format(output_str, k), 'wb') as f:
                    pickle.dump(lst_loss_his_g, f)
                with open('./record/loss_d_{}_{}.pkl'.format(output_str, k), 'wb') as f:
                    pickle.dump(lst_loss_his_d, f)
                img_sample = gan.sess.run(gan.g, feed_dict={gan.G_in_hair:b_tag_hair_fake3, 
                                                         gan.G_in_eyes:b_tag_eyes_fake3, 
                                                         gan.G_in_noise:ary_temp})
                scipy.misc.imsave('img/img_sample_{}_{}.jpg'.format(output_str,k),img_sample[0]) 
                print("Model saved in file: %s" % save_path)
    
            b_i += gan.bs
        
training()    

ary_tag_hair shape : (11568,)
ary_tag_eyes shape : (11568,)
ary_img shape : (11568, 64, 64, 3)
0
10368	acc : 0.226
1
10368	acc : 0.274
2
10368	acc : 0.366
3
10368	acc : 0.416
4
10368	acc : 0.394
5
10368	acc : 0.444
6
10368	acc : 0.428
7
10368	acc : 0.426
8
10368	acc : 0.428
9
10368	acc : 0.418
0
10368	acc : 0.248
1
10368	acc : 0.248
2
10368	acc : 0.344
3
10368	acc : 0.374
4
10368	acc : 0.39
5
10368	acc : 0.412
6
10368	acc : 0.384
7
10368	acc : 0.438
8
10368	acc : 0.438
9
10368	acc : 0.416
0
D loss : 10.282978057861328
G loss : -0.1346791535615921
saving model...


/nfs/nas-5.1/thchu/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:152: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Model saved in file: ./model_tf/model_gan9_bs16_HEclass_original_2.ckpt
D loss : 8.656083106994629
G loss : -0.5927639007568359
D loss : 6.8685526847839355
G loss : -1.3103978633880615
D loss : 2.4678211212158203
G loss : -3.703327178955078
D loss : -6.225380897521973
G loss : -11.144969940185547
D loss : -10.782952308654785
G loss : -30.037612915039062
D loss : -25.789569854736328
G loss : -77.09284973144531
D loss : -22.52804183959961
G loss : -141.54800415039062
D loss : -16.150760650634766
G loss : -120.32685852050781
D loss : -17.696094512939453
G loss : -83.56758117675781
D loss : -53.90604019165039
G loss : -71.4716796875
D loss : -54.74716567993164
G loss : -52.463645935058594
D loss : -111.36660766601562
G loss : 17.77989387512207
D loss : -178.67739868164062
G loss : -9.405557632446289
D loss : -428.8134765625
G loss : 57.68729782104492
D loss : -456.6224365234375
G loss : 106.26968383789062
D loss : -523.5787353515625
G loss : -93.41796875
D loss : -201.11497497558594
G loss

D loss : 514.9998168945312
G loss : -747.0230712890625
D loss : -151.32196044921875
G loss : -940.6925659179688
D loss : 570.7925415039062
G loss : -1684.50634765625
D loss : 91.48237609863281
G loss : -614.5416870117188
D loss : 646.8262939453125
G loss : 660.4102783203125
D loss : 367.95452880859375
G loss : 1821.98779296875
D loss : 422.2912292480469
G loss : -266.9112548828125
D loss : 159.04759216308594
G loss : -1584.956787109375
D loss : 356.50341796875
G loss : -1749.0970458984375
D loss : 74.4718017578125
G loss : -1742.7420654296875
D loss : 243.8710479736328
G loss : 1420.97509765625
D loss : 308.10626220703125
G loss : 492.30096435546875
D loss : 327.5150451660156
G loss : -119.13943481445312
D loss : 169.46461486816406
G loss : -1313.44189453125
D loss : -116.7326889038086
G loss : -23.21529769897461
D loss : -161.44570922851562
G loss : -202.2698974609375
D loss : 240.36016845703125
G loss : -1154.281494140625
D loss : 40.38686752319336
G loss : -1935.498779296875
D loss 

D loss : -63.51777267456055
G loss : 2299.509765625
D loss : 165.5795135498047
G loss : 1255.07080078125
D loss : 152.4681396484375
G loss : 37.008338928222656
D loss : 194.27926635742188
G loss : -327.150634765625
D loss : -48.6016960144043
G loss : -875.3851318359375
D loss : -239.9550323486328
G loss : -1331.5299072265625
D loss : -278.9176025390625
G loss : -1855.5015869140625
D loss : 10.334604263305664
G loss : -1224.619140625
D loss : -183.89056396484375
G loss : -663.2001953125
D loss : -86.50570678710938
G loss : -1215.56103515625
D loss : -81.70867919921875
G loss : -1011.6103515625
D loss : 138.50599670410156
G loss : -461.5142517089844
D loss : 30.07234001159668
G loss : -478.74395751953125
D loss : -159.81089782714844
G loss : 496.219482421875
D loss : 290.3668212890625
G loss : 1500.4078369140625
D loss : -1.0594844818115234
G loss : 1683.2152099609375
D loss : 150.12860107421875
G loss : 1183.5794677734375
D loss : -89.75062561035156
G loss : 1010.482177734375
D loss : -

...

D loss : 8.53062629699707
G loss : 41.76101303100586
D loss : 1.6978259086608887
G loss : 68.93792724609375
D loss : 3.8699021339416504
G loss : 56.77068328857422
D loss : 18.699893951416016
G loss : 20.91458511352539
D loss : 4.445939540863037
G loss : 7.4977498054504395
D loss : 16.26644515991211
G loss : -2.957671642303467
D loss : -1.0625314712524414
G loss : -16.050647735595703
D loss : 9.403352737426758
G loss : -19.27450942993164
D loss : -3.2078351974487305
G loss : -61.741966247558594
D loss : 7.350916385650635
G loss : -47.91022491455078
D loss : 5.380186557769775
G loss : 18.638900756835938
D loss : 5.141081809997559
G loss : 47.114681243896484
D loss : 11.63149642944336
G loss : 25.955108642578125
D loss : 77.89784240722656
G loss : 18.62982940673828
D loss : 18.790746688842773
G loss : 37.66870880126953
D loss : 6.702317714691162
G loss : 16.771329879760742
D loss : 4.489656925201416
G loss : 3.830984592437744
D loss : 11.849239349365234
G loss : 1.5652852058410645
D loss 

D loss : 7.136645793914795
G loss : -9.828475952148438
D loss : 0.1368083953857422
G loss : 38.18459701538086
D loss : -8.416563034057617
G loss : 64.740478515625
D loss : -27.79629135131836
G loss : 116.13336944580078
D loss : 39.2337646484375
G loss : 49.433013916015625
D loss : 3.4043869972229004
G loss : 82.68031311035156
D loss : 17.971424102783203
G loss : 74.29108428955078
D loss : 23.68213653564453
G loss : 74.99967193603516
D loss : 10.733003616333008
G loss : 78.77252197265625
D loss : 9.867670059204102
G loss : 89.94854736328125
D loss : 7.364006042480469
G loss : 89.69054412841797
D loss : 7.641124725341797
G loss : 74.65992736816406
D loss : 7.784689903259277
G loss : 50.235294342041016
D loss : 5.558449745178223
G loss : 31.581972122192383
D loss : 17.26218032836914
G loss : 12.518119812011719
D loss : -1.9081611633300781
G loss : -1.1540513038635254
D loss : -5.031618118286133
G loss : -25.181468963623047
D loss : 20.066570281982422
G loss : -29.11765480041504
D loss : 2

D loss : 12.80723762512207
G loss : 32.88531494140625
D loss : 12.425631523132324
G loss : 32.261383056640625
D loss : 7.934406280517578
G loss : 9.540931701660156
D loss : 2.21272611618042
G loss : 25.186813354492188
D loss : 7.875054836273193
G loss : 58.6923828125
D loss : 11.571898460388184
G loss : 46.779624938964844
D loss : 2.056399345397949
G loss : 35.420047760009766
D loss : 14.549077033996582
G loss : 29.138826370239258
D loss : 8.558443069458008
G loss : 13.087163925170898
D loss : 18.84269142150879
G loss : -3.858619213104248
D loss : 10.052099227905273
G loss : -18.27951431274414
D loss : 12.211575508117676
G loss : -10.43481159210205
D loss : 12.724689483642578
G loss : -30.759815216064453
D loss : 6.607393264770508
G loss : -32.92141342163086
D loss : 7.209780693054199
G loss : -47.84872817993164
D loss : 14.214683532714844
G loss : -36.50575256347656
D loss : 13.03731632232666
G loss : -7.632259845733643
D loss : 13.263614654541016
G loss : -3.0644607543945312
D loss :

D loss : 8.299609184265137
G loss : 29.33709716796875
D loss : 22.242490768432617
G loss : 43.36383056640625
D loss : 1.1847209930419922
G loss : 50.43235778808594
D loss : 5.99504280090332
G loss : 65.78820037841797
D loss : 7.9616007804870605
G loss : 77.56398010253906
D loss : 6.9806437492370605
G loss : 102.28033447265625
D loss : 10.756916046142578
G loss : 70.41078186035156
D loss : 12.34740161895752
G loss : 59.84373474121094
D loss : 11.866439819335938
G loss : 47.19313049316406
D loss : 11.157526969909668
G loss : 11.486440658569336
D loss : 3.6955394744873047
G loss : -8.750762939453125
D loss : 6.802883148193359
G loss : -15.366872787475586
D loss : 9.66313648223877
G loss : -46.67436218261719
D loss : 4.53683614730835
G loss : -37.50836181640625
D loss : 21.807947158813477
G loss : -40.983821868896484
D loss : 9.831147193908691
G loss : -39.108421325683594
D loss : 6.798544883728027
G loss : -39.285587310791016
D loss : 11.773355484008789
G loss : -24.609619140625
D loss : 

D loss : 7.822888374328613
G loss : 28.51529312133789
D loss : 7.467689514160156
G loss : 21.348854064941406
D loss : -5.572029113769531
G loss : 5.781357288360596
D loss : 11.614070892333984
G loss : 18.075923919677734
D loss : 12.191112518310547
G loss : -31.026039123535156
D loss : 8.160492897033691
G loss : -39.71106719970703
D loss : 5.855118751525879
G loss : -39.4921989440918
D loss : 4.80778694152832
G loss : -37.97461700439453
D loss : 15.061138153076172
G loss : -46.03437042236328
D loss : 10.6708984375
G loss : -26.970115661621094
D loss : 8.167801856994629
G loss : -29.55426788330078
D loss : 5.9436540603637695
G loss : -25.987403869628906
D loss : 8.811284065246582
G loss : -29.717679977416992
D loss : 6.819640159606934
G loss : -5.977262496948242


In [ ]:
# 1,6
    # 2,4,12
    # 3,5,9
    # 7,10
#     lst_hair = ['null', 1'orange hair', 2'white hair', 3'aqua hair', 4'gray hair',
#                 5'green hair', 6'red hair', 7'purple hair', 8'pink hair',
#                 9'blue hair', 10'black hair', 11'brown hair', 12'blonde hair']

In [ ]:
def test_show() :
    k = 0
    img_temp = skimage.io.imread('img_sample_{}_{}.jpg'.format(output_str,k))
    imshow(img_temp)
test_show()

In [ ]:
def text_pre(text) :
    text = text.replace(',',' ')
    lst_text = text.split(' ')
    hair_i = 0
    eyes_i = 0
    for i,s in enumerate(lst_text) :
        if s == 'hair' :
            hair_i = i
        elif s == 'eyes' :
            eyes_i = i
    if hair_i :
        hair_style = lst_text[hair_i-1] + ' ' + lst_text[hair_i]
    else :
        hair_style = 'null'
    if eyes_i :
        eyes_style = lst_text[eyes_i-1] + ' ' + lst_text[eyes_i]
    else :
        eyes_style = 'null'
    
    return hair_style, eyes_style

## testing

## special task

In [ ]:
def generate_img() :
    for i in range(5) :
        ary_img = np.random.randint(255,size=(64,64,3))
        scipy.misc.imsave('sample_1_{}.jpg'.format(i+1),ary_img)

In [ ]:
text = 'red hair, green eyes'
# text = 'red eyes'
hair_style, eyes_style = text_pre(text)
print (hair_style)
print (eyes_style)

generate_img()

In [ ]:
start_t = time.time()
args = get_args()
model_type = 'CNN'
path_model = get_path_new_model(model_type)
#path_model = './model/CNN_1.h5'


print ('all process cost {} seconds'.format(time.time() - start_t))
print ('all done')

In [ ]:
a = np.arange(15).reshape((3,5))
print (a[[0,2]])


In [ ]:
# 'value' is a tensor with shape [5, 30]
# Split 'value' into 3 tensors with sizes [4, 15, 11] along dimension 1
v = tf.constant(1, shape=[5,30])
v2 = tf.constant(2, shape=[5,30])
vv = tf.add(v,v2)
sess = tf.Session(config=config)
vv = sess.run(vv, feed_dict={})
print(vv)

print (v)
s0, s1, s2 = tf.split(v, [1, 1, 3], 0)
print (s0)
print (s2)

In [ ]:
a = np.arange(15).reshape((3,5))
b = np.ones((3,5))

print ((a+b)/2)

c = tf.placeholder(tf.float32, shape=(None,3,5))
c = tf.constant(1.0, shape=[3,5])
cc = c[:2] + 1
print (cc)
x = tf.range(0,15,1,dtype=tf.float32)
xx = tf.reshape(x,shape=[3,5])


d = tf.norm(c, ord=2)
print (d)
sess = tf.Session(config=config)
vv = sess.run(xx+c, feed_dict={})
print (vv)

In [ ]:
np.random.normal(0,1,[3,5])